***GENERATED CODE FOR glassdoorjobssalaryregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Job Title', 'transformation_label': 'String Indexer'}], 'feature': 'Job Title', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'AG DATA SCIENTIST', 'max': 'WEB DATA ANALYST', 'missing': '0', 'distinct': '206'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Job Title'}, {'feature_label': 'Job Title', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Job Title')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Salary Estimate', 'transformation_label': 'String Indexer'}], 'feature': 'Salary Estimate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$10-$17 Per Hour', 'max': '$99K-$157K ', 'missing': '0', 'distinct': '344'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Salary Estimate'}, {'feature_label': 'Salary Estimate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Salary Estimate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Company Name', 'transformation_label': 'String Indexer'}], 'feature': 'Company Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1-800-FLOWERS.COM, Inc.', 'max': 'webfx.com', 'missing': '0', 'distinct': '286'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Company Name'}, {'feature_label': 'Company Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Company Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Size', 'transformation_label': 'String Indexer'}], 'feature': 'Size', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 to 50 employees', 'max': '51 to 200 employees', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Size'}, {'feature_label': 'Size', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Size')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Type of ownership', 'transformation_label': 'String Indexer'}], 'feature': 'Type of ownership', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'College / University', 'max': 'Subsidiary or Business Segment', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Type of ownership'}, {'feature_label': 'Type of ownership', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Type of ownership')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Industry', 'transformation_label': 'String Indexer'}], 'feature': 'Industry', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Advertising & Marketing', 'max': 'Wholesale', 'missing': '0', 'distinct': '55'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Industry'}, {'feature_label': 'Industry', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Industry')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Revenue', 'transformation_label': 'String Indexer'}], 'feature': 'Revenue', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '$1 to $2 billion (USD)', 'max': 'Unknown / Non-Applicable ', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Revenue'}, {'feature_label': 'Revenue', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Revenue')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'job_state', 'transformation_label': 'String Indexer'}], 'feature': 'job_state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' AL', 'max': ' WI', 'missing': '0', 'distinct': '37'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'job_state'}, {'feature_label': 'job_state', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('job_state')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run glassdoorjobssalaryregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	glassdoorsalarycleaned = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Glassdoor_Salary_Cleaned.csv', 'filename': 'Glassdoor_Salary_Cleaned.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run glassdoorjobssalaryregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	glassdoorjobssalaryregressionautofe = TransformationMain.run(glassdoorsalarycleaned,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Job Title", "transformation_label": "String Indexer"}], "feature": "Job Title", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AG DATA SCIENTIST", "max": "WEB DATA ANALYST", "missing": "0", "distinct": "206"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Job Title"}, {"transformationsData": [{"feature_label": "Salary Estimate", "transformation_label": "String Indexer"}], "feature": "Salary Estimate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$10-$17 Per Hour", "max": "$99K-$157K ", "missing": "0", "distinct": "344"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Salary Estimate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Rating", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.73", "stddev": "0.55", "min": "1.9", "max": "5.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Rating"}, {"transformationsData": [{"feature_label": "Company Name", "transformation_label": "String Indexer"}], "feature": "Company Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1-800-FLOWERS.COM, Inc.", "max": "webfx.com", "missing": "0", "distinct": "286"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Company Name"}, {"transformationsData": [{"feature_label": "Size", "transformation_label": "String Indexer"}], "feature": "Size", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 to 50 employees", "max": "51 to 200 employees", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Size"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Founded", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1969.56", "stddev": "55.58", "min": "1744", "max": "2019", "missing": "0"}, "updatedLabel": "Founded"}, {"transformationsData": [{"feature_label": "Type of ownership", "transformation_label": "String Indexer"}], "feature": "Type of ownership", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "College / University", "max": "Subsidiary or Business Segment", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Type of ownership"}, {"transformationsData": [{"feature_label": "Industry", "transformation_label": "String Indexer"}], "feature": "Industry", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Advertising & Marketing", "max": "Wholesale", "missing": "0", "distinct": "55"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Industry"}, {"transformationsData": [{"feature_label": "Revenue", "transformation_label": "String Indexer"}], "feature": "Revenue", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "$1 to $2 billion (USD)", "max": "Unknown / Non-Applicable ", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Revenue"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hourly", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "hourly"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "employer_provided", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.06", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "employer_provided"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "min_salary", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "74.96", "stddev": "31.76", "min": "10", "max": "202", "missing": "0"}, "updatedLabel": "min_salary"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "max_salary", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "128.75", "stddev": "47.17", "min": "16", "max": "306", "missing": "0"}, "updatedLabel": "max_salary"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "avg_salary", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "101.85", "stddev": "38.97", "min": "13.5", "max": "254.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "avg_salary"}, {"transformationsData": [{"feature_label": "job_state", "transformation_label": "String Indexer"}], "feature": "job_state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " AL", "max": " WI", "missing": "0", "distinct": "37"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "job_state"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "same_state", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.58", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "same_state"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "50.44", "stddev": "55.58", "min": "1", "max": "276", "missing": "0"}, "updatedLabel": "age"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "python_yn", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.57", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "python_yn"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "R_yn", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.06", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "R_yn"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "spark", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.25", "stddev": "0.43", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "spark"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "aws", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.23", "stddev": "0.42", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "aws"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "excel", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.51", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "excel"}]}))

	#transformationPostExecutionHook(glassdoorjobssalaryregressionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run glassdoorjobssalaryregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(glassdoorjobssalaryregressionautofe, ["Rating", "Founded", "hourly", "employer_provided", "min_salary", "max_salary", "same_state", "age", "python_yn", "R_yn", "spark", "aws", "excel", "Job Title_stringindexer", "Salary Estimate_stringindexer", "Company Name_stringindexer", "Size_stringindexer", "Type of ownership_stringindexer", "Industry_stringindexer", "Revenue_stringindexer", "job_state_stringindexer"], "avg_salary")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

